In [1]:
ROBBER = "R"
POLICE = "P"
BANK = "B"
EMPTY = "*"

grid = [
    [ROBBER,EMPTY,EMPTY,EMPTY],
    [EMPTY,BANK,EMPTY,EMPTY],
    [EMPTY,EMPTY,EMPTY,EMPTY],
    [EMPTY,EMPTY,EMPTY, POLICE]
]

for row in grid:
    print(' '.join(row))

R * * *
* B * *
* * * *
* * * P


In [2]:
class State:
    
    def __init__(self, grid, robber_pos, police_pos):
        self.grid = grid
        self.robber_pos = robber_pos
        self.police_pos = police_pos
        
    def __eq__(self, other):
        return isinstance(other, State) and self.grid == other.grid and self.robber_pos == other.robber_pos
    
    def __hash__(self):
        return hash(str(self.grid) + str(self.robber_pos))
    
    def __str__(self):
        return f"State(grid={self.grid}, robber_pos={self.robber_pos})"

In [3]:
STAY = 0
UP = 1
DOWN = 2 
LEFT = 3
RIGHT = 4

ACTIONS = [STAY, UP, DOWN, LEFT, RIGHT]

REWARD_BANK = 1
REWARD_EMPTY = -1
REWARD_CAUGHT = -10

START_ROBBER = [0,0]
START_POLICE = [3,3]

In [4]:
start_state = State(grid=grid, robber_pos=START_ROBBER, police_pos=START_POLICE)

In [5]:
from copy import deepcopy

def act(state,action_robber,action_police):
    # state_actor_pos = state.robber_pos for robber or state.police_pos
    def new_actor_pos(state,state_actor_pos,action):
        p = deepcopy(state_actor_pos)
        if action == UP:
            #print('UP')
            p[0] = max(0,p[0]-1)
        elif action == DOWN:
            #print('down')
            p[0] = min(len(state.grid)-1, p[0]+1)
        elif action == LEFT:
            #print('LEFT')
            p[1] = max(0, p[1]-1)
        elif action == RIGHT:
            #print('RIGHT')
            p[1] = min(len(state.grid[0])-1, p[1]+1)
        elif state_actor_pos == deepcopy(state.robber_pos):
            if action == STAY:
                #print('STAY')
                p = p
            else:
                #print('error robber')
                raise ValueError(f"Unknown action {action}")
        else:
            #print('error police')
            raise ValueError(f"Unknown action {action}")
        return p
    
    pos_robber = new_actor_pos(state,state.robber_pos,action_robber)
    pos_police = new_actor_pos(state,state.police_pos,action_police)
    
    new_grid = deepcopy(state.grid)
    
    old_pos_robber = state.robber_pos
    old_pos_police = state.police_pos
    
    grid_item_robber = state.grid[pos_robber[0]][pos_robber[1]]
    grid_item_police = state.grid[pos_police[0]][pos_police[1]]
    old_grid_item_robber = state.grid[old_pos_robber[0]][old_pos_robber[1]]
    old_grid_item_police = state.grid[old_pos_police[0]][old_pos_police[1]]
    
    if pos_robber == pos_police:
        #print('1')
        #print('OVER')
        reward = REWARD_CAUGHT
        is_over = False
        if not old_pos_robber == pos_robber:
            if old_grid_item_robber == BANK+ROBBER:
                new_grid[old_pos_robber[0]][old_pos_robber[1]] = BANK
                new_grid[pos_robber[0]][pos_robber[1]] = ROBBER
            elif grid_item_robber == BANK:
                new_grid[old_pos_robber[0]][old_pos_robber[1]] = EMPTY
                new_grid[pos_robber[0]][pos_robber[1]] = BANK+ROBBER 
            else:
                new_grid[old_pos_robber[0]][old_pos_robber[1]] = EMPTY
                new_grid[pos_robber[0]][pos_robber[1]] = ROBBER
        if old_pos_robber == pos_robber:
            #print('balla')
            if old_grid_item_robber == BANK+ROBBER:
                new_grid[pos_robber[0]][pos_robber[1]] = BANK+ROBBER
            else:
                new_grid[pos_robber[0]][pos_robber[1]] = ROBBER
        if old_grid_item_police == BANK+POLICE:
            new_grid[old_pos_police[0]][old_pos_police[1]] = BANK
            new_grid[pos_police[0]][pos_police[1]] += POLICE
        else:
            new_grid[old_pos_police[0]][old_pos_police[1]] = EMPTY
            new_grid[pos_police[0]][pos_police[1]] += POLICE
            
    elif old_pos_robber == old_pos_police:
        #print('2')
        reward = REWARD_EMPTY
        is_over = True
        if old_grid_item_robber == BANK+ROBBER+POLICE:
            new_grid[old_pos_robber[0]][old_pos_robber[1]] = BANK
        else:
            new_grid[old_pos_robber[0]][old_pos_robber[1]] = EMPTY
        new_grid[START_ROBBER[0]][START_ROBBER[1]] = ROBBER
        new_grid[START_POLICE[0]][START_POLICE[1]] = POLICE
        pos_police = START_POLICE
        pos_robber = START_ROBBER
        new_grid = deepcopy(start_state.grid)
    
    elif pos_robber == old_pos_police and pos_police == old_pos_robber:
        #print('3')
        if grid_item_robber == BANK+POLICE:
            reward = REWARD_BANK
            is_over = False
            new_grid[old_pos_robber[0]][old_pos_robber[1]] = POLICE
            new_grid[old_pos_police[0]][old_pos_police[1]] = BANK+ROBBER
        elif grid_item_police == BANK+ROBBER:
            reward = REWARD_EMPTY
            is_over = False
            new_grid[old_pos_robber[0]][old_pos_robber[1]] = BANK+POLICE
            new_grid[old_pos_police[0]][old_pos_police[1]] = ROBBER
        else:
            reward = REWARD_EMPTY
            is_over = False
            new_grid[old_pos_robber[0]][old_pos_robber[1]] = POLICE
            new_grid[old_pos_police[0]][old_pos_police[1]] = ROBBER
    
    elif pos_police == old_pos_robber and pos_robber != old_pos_police:
        #print('4')
        if old_grid_item_police == BANK+POLICE:
            reward = REWARD_EMPTY
            is_over = False
            new_grid[old_pos_police[0]][old_pos_police[1]] = BANK
            new_grid[pos_police[0]][pos_police[1]] = POLICE
            new_grid[pos_robber[0]][pos_robber[1]] = ROBBER
        elif old_grid_item_robber == BANK+ROBBER:
            reward = REWARD_EMPTY
            is_over = False
            new_grid[old_pos_police[0]][old_pos_police[1]] = EMPTY
            new_grid[pos_police[0]][pos_police[1]] = BANK+POLICE
            new_grid[pos_robber[0]][pos_robber[1]] = ROBBER
        elif grid_item_robber == BANK:
            reward = REWARD_BANK
            is_over = False
            new_grid[old_pos_police[0]][old_pos_police[1]] = EMPTY
            new_grid[pos_police[0]][pos_police[1]] = POLICE
            new_grid[pos_robber[0]][pos_robber[1]] = BANK+ROBBER
        else:
            reward = REWARD_EMPTY
            is_over = False
            new_grid[old_pos_police[0]][old_pos_police[1]] = EMPTY
            new_grid[pos_police[0]][pos_police[1]] = POLICE
            new_grid[pos_robber[0]][pos_robber[1]] = ROBBER
    
    elif pos_robber == old_pos_police and pos_police != old_pos_robber:
        #print('5')
        if old_grid_item_robber == BANK+ROBBER:
            reward = REWARD_EMPTY
            is_over = False
            new_grid[old_pos_robber[0]][old_pos_robber[1]] = BANK
            new_grid[pos_robber[0]][pos_robber[1]] = ROBBER
            new_grid[pos_police[0]][pos_police[1]] = POLICE
        elif old_grid_item_police == BANK+POLICE:
            reward = REWARD_BANK
            is_over = False
            new_grid[old_pos_robber[0]][old_pos_robber[1]] = EMPTY
            new_grid[pos_robber[0]][pos_robber[1]] = BANK+ROBBER
            new_grid[pos_police[0]][pos_police[1]] = POLICE
        elif grid_item_police == BANK:
            reward = REWARD_EMPTY
            is_over = False
            new_grid[old_pos_robber[0]][old_pos_robber[1]] = EMPTY
            new_grid[pos_robber[0]][pos_robber[1]] = ROBBER
            new_grid[pos_police[0]][pos_police[1]] = BANK+POLICE
        else:
            reward = REWARD_EMPTY
            is_over = False
            new_grid[old_pos_robber[0]][old_pos_robber[1]] = EMPTY
            new_grid[pos_robber[0]][pos_robber[1]] = ROBBER
            new_grid[pos_police[0]][pos_police[1]] = POLICE
        
    else:
        #print('6')
        if grid_item_police == BANK:
            #print("Polis bank")
            reward = REWARD_EMPTY
            is_over = False
            new_grid[pos_police[0]][pos_police[1]] = BANK+POLICE
            new_grid[old_pos_police[0]][old_pos_police[1]] = EMPTY
            new_grid[pos_robber[0]][pos_robber[1]] = ROBBER
            if old_pos_robber != pos_robber:
                new_grid[old_pos_robber[0]][old_pos_robber[1]] = EMPTY
        elif grid_item_robber == BANK:
            reward = REWARD_BANK
            is_over = False
            new_grid[pos_robber[0]][pos_robber[1]] = BANK+ROBBER
            new_grid[old_pos_robber[0]][old_pos_robber[1]] = EMPTY
            new_grid[pos_police[0]][pos_police[1]] = POLICE
            new_grid[old_pos_police[0]][old_pos_police[1]] = EMPTY
        elif grid_item_robber == BANK+ROBBER:
            reward = REWARD_BANK
            is_over = False
            new_grid[pos_robber[0]][pos_robber[1]] = BANK+ROBBER
            new_grid[pos_police[0]][pos_police[1]] = POLICE
            new_grid[old_pos_police[0]][old_pos_police[1]] = EMPTY
        elif old_grid_item_police == BANK+POLICE:
            reward = REWARD_EMPTY
            is_over = False
            new_grid[pos_police[0]][pos_police[1]] = POLICE
            new_grid[old_pos_police[0]][old_pos_police[1]] = BANK
            new_grid[pos_robber[0]][pos_robber[1]] = ROBBER
            if old_pos_robber != pos_robber:
                new_grid[old_pos_robber[0]][old_pos_robber[1]] = EMPTY
        elif grid_item_robber == ROBBER:
            #print('HÄÄR')
            reward = REWARD_EMPTY
            is_over = False
            new_grid[pos_robber[0]][pos_robber[1]] = ROBBER
            new_grid[pos_police[0]][pos_police[1]] = POLICE
            new_grid[old_pos_police[0]][old_pos_police[1]] = EMPTY
        elif old_grid_item_robber == BANK+ROBBER:
            reward = REWARD_EMPTY
            is_over = False
            new_grid[pos_robber[0]][pos_robber[1]] = ROBBER
            new_grid[old_pos_robber[0]][old_pos_robber[1]] = BANK
            new_grid[pos_police[0]][pos_police[1]] = POLICE
            new_grid[old_pos_police[0]][old_pos_police[1]] = EMPTY
        else:
            reward = REWARD_EMPTY
            is_over = False
            new_grid[pos_robber[0]][pos_robber[1]] = ROBBER
            if old_pos_robber != pos_robber:
                new_grid[old_pos_robber[0]][old_pos_robber[1]] = EMPTY
            new_grid[pos_police[0]][pos_police[1]] = POLICE
            new_grid[old_pos_police[0]][old_pos_police[1]] = EMPTY
    
  
    return State(grid=new_grid,robber_pos=pos_robber, police_pos=pos_police), reward, is_over

In [6]:
import numpy as np
import random
r=random.seed(42)

N_STATES = 16*16
ITERATIONS = 1000

eps = 0.2
LAMBDA = 0.8

q_table = dict()

In [7]:
def q(state, action_robber=None):
    
    if state not in q_table:
        q_table[state] = np.zeros(len(ACTIONS))
    if action_robber is None: 
        return q_table[state]
    
    return q_table[state][action_robber]



In [8]:
def choose_action(state):
    pos_robber = state.robber_pos
    pos_police = state.police_pos
    if pos_police[0] == 0:
        if pos_police[1] == 0:
            action_police = random.choice([ACTIONS[2],ACTIONS[4]])
        elif pos_police[1] == 3:
            action_police = random.choice([ACTIONS[2],ACTIONS[3]])
        else:
            action_police = random.choice(ACTIONS[2:5])
    elif pos_police[0] == 3:
        if pos_police[1] == 0:
            action_police = random.choice([ACTIONS[1],ACTIONS[4]])
        elif pos_police[1] == 3:
            action_police = random.choice([ACTIONS[1],ACTIONS[3]])
        else:
            action_police = random.choice([ACTIONS[1],ACTIONS[3],ACTIONS[4]])
    elif pos_police[1] == 0:
        action_police = random.choice([ACTIONS[1],ACTIONS[2],ACTIONS[4]])
    elif pos_police[1] == 3:
        action_police = random.choice(ACTIONS[1:4])
    else: 
        action_police = random.choice(ACTIONS[1:])
        
    if random.uniform(0, 1) < eps:   
        if pos_robber[0] == 0:
            if pos_robber[1] == 0:
                action_robber = random.choice([ACTIONS[0],ACTIONS[2],ACTIONS[4]])
            elif pos_robber[1] == 3:
                action_robber = random.choice([ACTIONS[0],ACTIONS[2],ACTIONS[3]])
            else:
                action_robber = random.choice([ACTIONS[0],ACTIONS[2],ACTIONS[3],ACTIONS[4]])
        elif pos_robber[0] == 3:
            if pos_robber[1] == 0:
                action_robber = random.choice([ACTIONS[0],ACTIONS[1],ACTIONS[4]])
            elif pos_robber[1] == 3:
                action_robber = random.choice([ACTIONS[0],ACTIONS[1],ACTIONS[3]])
            else:
                action_robber = random.choice([ACTIONS[0],ACTIONS[1],ACTIONS[3],ACTIONS[4]])
        elif pos_robber[1] == 0:
            action_robber = random.choice([ACTIONS[0],ACTIONS[1],ACTIONS[2],ACTIONS[4]])
        elif pos_robber[1] == 3:
            action_robber = random.choice(ACTIONS[0:4])
        else: 
            action_robber = random.choice(ACTIONS)
    else:
        action_robber = np.argmax(q(state))
    
    return action_robber, action_police


In [9]:
state = start_state
total_reward = 0

values = []
times = []
for t in range(ITERATIONS):
    
    alpha = 1/(1+t**(2/3))
    action_robber, action_police = choose_action(state)
    next_state, reward, over = act(state, action_robber, action_police)
    total_reward += reward
    #if over:
    #    total_reward = 0;

    q(state)[action_robber] = q(state, action_robber) + \
            alpha * (reward + LAMBDA *  np.max(q(next_state)) - q(state, action_robber))

    state = next_state
    if t%1000==0:
        #print(t)
        #print(q(state)[action_robber])
        values.append(q(state)[action_robber])
        times.append(t)
    #if t == ITERATIONS:
    #    break
    #print(f"total reward -> {total_reward}")

In [10]:
%matplotlib inline
import matplotlib.pyplot as plt

plt.plot(times,values,'b')
plt.show()

99.99999999999997